In [ ]:
!pip install -U faker --quiet
%pip install schwifty

In [1]:
import pandas as pd
import datetime
import random
import uuid
pd.options.mode.chained_assignment = None

In [2]:
# Necessary imports for this notebook
import os

import numpy as np
import pandas as pd

import datetime
import time

import random

# For plotting
%matplotlib inline

import matplotlib.pyplot as plt

In [5]:
from schwifty import BIC
from schwifty import IBAN
import numpy as np

required_coutry = pd.read_csv('./static/countries.csv')
required_banks = pd.read_csv('./static/banks.csv',header=None)
required_banks = pd.read_json('./static/generated_de.json')
required_banks = pd.read_json('./static/manual_fr.json')

required_coutries = required_coutry['alpha-2']
required_banks = required_banks['bank_code']

import sys;
print(sys.version)
print(sys.path)

import dep.random_names as random_names


def generate_customer_profiles_table(n_customers, random_state=0):
    
    np.random.seed(random_state)
        
    customer_id_properties=[]
    
    # Generate customer properties from random distributions 
    for customer_id in range(n_customers):
        
        x_customer_id = np.random.uniform(0,100)
        y_customer_id = np.random.uniform(0,100)
        
        mean_amount = np.random.uniform(5,150) # Arbitrary (but sensible) value 
        std_amount = mean_amount/2 # Arbitrary (but sensible) value
        
        mean_nb_tx_per_day = np.random.uniform(0,4) # Arbitrary (but sensible) value 
        
        country = required_coutries.sample(n=1).to_list()
        country.append('FR')
        country = random.choice(country)
        bank_code_sample = str(required_banks.sample(n=1).to_list()[0])
        account_code_sample = ''.join([str(i) for i in range(1,9)])

        name = random_names.Full()         # outputs random first and last name

        try:
            iban = IBAN.generate(country, bank_code=bank_code_sample, account_code=account_code_sample)
            bic = BIC.from_bank_code(country,bank_code_sample)
        except Exception as err:
            iban = err
            bic = err

        customer_id_properties.append([customer_id,
                                      x_customer_id, y_customer_id,
                                      mean_amount, std_amount,
                                      mean_nb_tx_per_day,iban,bic,name,country])
        
    customer_profiles_table = pd.DataFrame(customer_id_properties, columns=['CUSTOMER_ID',
                                                                      'x_customer_id', 'y_customer_id',
                                                                      'mean_amount', 'std_amount',
                                                                      'mean_nb_tx_per_day','iban','bic','fullname','country'])
    
    return customer_profiles_table

generate_customer_profiles_table(100).head(10)

3.9.5 (default, Jun  4 2021, 12:28:51) 
[GCC 7.5.0]
['/home/simon/local_branch/AML/AML/src/LCB-PRJ', '/home/simon/miniconda3/lib/python39.zip', '/home/simon/miniconda3/lib/python3.9', '/home/simon/miniconda3/lib/python3.9/lib-dynload', '', '/home/simon/miniconda3/lib/python3.9/site-packages']


,CUSTOMER_ID,x_customer_id,y_customer_id,mean_amount,std_amount,mean_nb_tx_per_day,iban,bic,fullname,country
0,0,54.881350,71.518937,92.400690,46.200345,2.179533,FR7613507000000001234567886,CCBPFRPPLIL,Hendrik Dailey,FR
1,1,81.552382,15.941446,96.190274,48.095137,1.593737,FR7613259000000001234567842,KOLBFR21XXX,Rowley Willet,FR
2,2,72.143011,48.289162,33.133308,16.566654,0.822265,FR7630001000000001234567821,BDFEFRPPCCT,Dobb Vaasta,FR
3,3,43.553154,79.868925,138.901053,69.450527,1.196615,Bank code exceeds maximum size 4,Bank code exceeds maximum size 4,Eglanteen Towrey,SK
4,4,23.370848,76.252471,146.904900,73.452450,2.534573,FR7630066000000001234567856,CMCIFRPPXXX,Alcina Adamina,FR
5,5,68.133567,10.390146,114.828443,57.414222,1.963824,FR7610278000000001234567856,CMCIFRPAXXX,Almira Zito,FR
6,6,59.336552,14.508289,115.737749,57.868874,3.411490,FR7614506000000001234567827,AGRIFRPP845,Olethea Anana,FR
7,7,15.805973,66.508162,112.068354,56.034177,2.352807,FR7612548000000001234567877,AXABFRPPXXX,Yursa Greenberg,FR
8,8,78.901425,69.266165,62.930465,31.465232,0.307190,Unknown bank code '30004' for country 'DE',Unknown bank code '30004' for country 'DE',Djordje Rabipour,DE
9,9,0.302650,42.839512,91.075212,45.537606,3.904806,Bank code exceeds maximum size 3,Bank code exceeds maximum size 3,Jair Prickett,CY


In [6]:
def generate_terminal_profiles_table(n_terminals, random_state=0):
    
    np.random.seed(random_state)
        
    terminal_id_properties=[]
    
    # Generate terminal properties from random distributions 
    for terminal_id in range(n_terminals):
        
        x_terminal_id = np.random.uniform(0,100)
        y_terminal_id = np.random.uniform(0,100)
        
        terminal_id_properties.append([terminal_id,
                                      x_terminal_id, y_terminal_id])
                                       
    terminal_profiles_table = pd.DataFrame(terminal_id_properties, columns=['TERMINAL_ID',
                                                                      'x_terminal_id', 'y_terminal_id'])
    
    return terminal_profiles_table

In [7]:
def get_list_terminals_within_radius(customer_profile, x_y_terminals, r):
    
    # Use numpy arrays in the following to speed up computations
    
    # Location (x,y) of customer as numpy array
    x_y_customer = customer_profile[['x_customer_id','y_customer_id']].values.astype(float)
    
    # Squared difference in coordinates between customer and terminal locations
    squared_diff_x_y = np.square(x_y_customer - x_y_terminals)
    
    # Sum along rows and compute suared root to get distance
    dist_x_y = np.sqrt(np.sum(squared_diff_x_y, axis=1))
    
    # Get the indices of terminals which are at a distance less than r
    available_terminals = list(np.where(dist_x_y<r)[0])
    
    # Return the list of terminal IDs
    return available_terminals
    

In [8]:
def get_list_beneficiary_within_date(customer_profile, ext_list):

    low = 1
    n_int = np.random.randint(low, 50)

    # Get the indices of terminals which are at a distance less than r
    available_beneficiaries = ext_list.sample(n=n_int)[['CUSTOMER_ID','iban','bic','fullname','country']]
    colNames = {str(col): "ext_"+ str(col).lower() for col in available_beneficiaries.columns}
    available_beneficiaries = available_beneficiaries.rename(columns=colNames)
    available_beneficiaries = available_beneficiaries.to_dict('records')[0] #.to_dict('index')

    available_beneficiaries = list(ext_list.sample(n=n_int).index)

    # Return the list of customer IDs
    return available_beneficiaries

In [9]:
def generate_transactions_table(customer_profile, start_date = "2022-11-04", nb_days = 1):
    
    customer_transactions = []
    
    random.seed(int(customer_profile.CUSTOMER_ID))
    np.random.seed(int(customer_profile.CUSTOMER_ID))
    
    # For all days
    for day in range(nb_days):
        
        # Random number of transactions for that day 
        nb_tx = np.random.poisson(customer_profile.mean_nb_tx_per_day)
        
        # If nb_tx positive, let us generate transactions
        if nb_tx>0:
            
            for tx in range(nb_tx):
                
                # Time of transaction: Around noon, std 20000 seconds. This choice aims at simulating the fact that 
                # most transactions occur during the day.
                time_tx = int(np.random.normal(86400/2, 20000))
                
                # If transaction time between 0 and 86400, let us keep it, otherwise, let us discard it
                if (time_tx>0) and (time_tx<86400):
                    
                    # Amount is drawn from a normal distribution  
                    amount = np.random.normal(customer_profile.mean_amount, customer_profile.std_amount)
                    
                    # If amount negative, draw from a uniform distribution
                    if amount<0:
                        amount = np.random.uniform(0,customer_profile.mean_amount*2)
                    
                    amount=np.round(amount,decimals=2)
                    
                    if len(customer_profile.available_terminals)>0:
                        
                        terminal_id = random.choice(customer_profile.available_terminals)
                    
                        customer_transactions.append([time_tx+day*86400, day,
                                                      customer_profile.CUSTOMER_ID, 
                                                      terminal_id, amount])
            
    customer_transactions = pd.DataFrame(customer_transactions, columns=['TX_TIME_SECONDS', 'TX_TIME_DAYS', 'CUSTOMER_ID', 'TERMINAL_ID', 'TX_AMOUNT'])
    
    if len(customer_transactions)>0:
        customer_transactions['TX_DATETIME'] = pd.to_datetime(customer_transactions["TX_TIME_SECONDS"], unit='s', origin=start_date)
        customer_transactions=customer_transactions[['TX_DATETIME','CUSTOMER_ID', 'TERMINAL_ID', 'TX_AMOUNT','TX_TIME_SECONDS', 'TX_TIME_DAYS']]
    
    return customer_transactions

In [10]:
import json 
data = '''{
	"payinId":"aeeab4bb-4f8a-5890-9451-5da4133ac369",
	"payinTag":"RECEPTION_82de1263-7903-11ec-9650-c6992779932e|82de1263-7903-11ec-9650-c6992779932e",
	"walletId":"1226291",
	"userId":"3",
	"payinStatus":"VALIDATED",
	"paymentMethodId":"27",
	"messageToUser":"",
	"subtotalItems":"",
	"subtotalServices":"",
	"subtotalTax":"",
	"amount":"10.00",
	"currency":"EUR",
	"createdDate":"2022-01-19 08:40:54",
	"walletEventName":"Event_test",
	"walletAlias":"event-test-61e7cc51e6f4b",
	"userFirstname":"Alex",
	"userLastname":"Hond",
	"codeStatus":"140005",
	"informationStatus":"",
	"refundAmount":"",
	"ibanFullname":"PIERRE XXXXX",
	"DbtrIBAN":"FR7630003045100002009351319",
	"ibanBic":"SOGEFRPPXXX",
	"ibanTxEndToEndId":"82de1248-7903-11ec-9650-c6992779932e",
	"ibanTxId":"82de1263-7903-11ec-9650-c6992779932e",
	"forwardUrl":"",
	"paymentAcceptedUrl":"",
	"paymentRefusedUrl":"",
	"paymentWaitingUrl":"",
	"paymentExceptionUrl":"",
	"paymentCanceledUrl":"",
	"payinDate":"",
	"mandateId":"0",
	"creditorName":"John Smith",
	"creditorAddressLine":"99 RUE XXXXXXX PARIS 75003",
	"creditorCountry":"FR",
	"creditorIban":"FR7616798000010000122629170",
	"creditorBIC":"TRZOFR21XXX",
	"virtualIbanId":"",
	"virtualIbanReference":"",
	"additionalData":"",
	"ibanId":"d3e3e053f08fa2b200971adf2b481cec0c6ab369"
}'''

sample_api = json.loads(data)

sample_df = pd.DataFrame(sample_api, index=[0])

df = pd.json_normalize(
    sample_api
)

df_t = df.T
temp = df_t[df_t[0] != ""]
temp
#temp.T.columns

import uuid
payinId = uuid.uuid4()
ibanTxId = uuid.uuid4()
ibanId = uuid.uuid4()

tx_direction = pd.DataFrame([['debit'],['credit']],columns=['direction'])
tx_status =  pd.DataFrame(['CANCELLED','VALIDATED','PENDING'],columns=['tx_status'])
tx_sepa_scheme =  pd.DataFrame(['SCTR','SCTE','SDDR','SDDE'],columns=['tx_sepa_scheme'])

In [11]:
def generate_sepa_trx_table(customer_profile, ext_customer_profile, start_date = "2022-11-04", nb_days = 1):
    
    customer_transactions = []
    
    random.seed(int(customer_profile.CUSTOMER_ID))
    np.random.seed(int(customer_profile.CUSTOMER_ID))

    #random.seed(int(ext_customer_profile.CUSTOMER_ID))
    #np.random.seed(int(ext_customer_profile.CUSTOMER_ID))
    tx_direction = pd.DataFrame([['debit'],['credit']],columns=['direction'])
    tx_status =  pd.DataFrame([['CANCELLED'],['VALIDATED'],['PENDING']],columns=['status'])
    tx_sepa_scheme =  pd.DataFrame([['SCTR'],['SCTE'],['SDDR'],['SDDE']],columns=['scheme'])
    tx_code_status = [140005]

    # For all days
    for day in range(nb_days):
        
        # Random number of transactions for that day 
        nb_tx = np.random.poisson(customer_profile.mean_nb_tx_per_day)
        
        # If nb_tx positive, let us generate transactions
        if nb_tx>0:
            
            for tx in range(nb_tx):
                
                # Time of transaction: Around noon, std 20000 seconds. This choice aims at simulating the fact that 
                # most transactions occur during the day.
                time_tx = int(np.random.normal(86400/2, 20000))
                
                # If transaction time between 0 and 86400, let us keep it, otherwise, let us discard it
                if (time_tx>0) and (time_tx<86400):
                    
                    # Amount is drawn from a normal distribution  
                    amount = np.random.normal(customer_profile.mean_amount, customer_profile.std_amount) * 10 * random.randint(1,7) # SEPA TRX
                    
                    # If amount negative, draw from a uniform distribution
                    if amount<0:
                        amount = np.random.uniform(0,customer_profile.mean_amount*2)
                    
                    amount=np.round(amount,decimals=2)
                    
                    if len(customer_profile.available_terminals)>0:
                        
                        terminal_id = random.choice(customer_profile.available_terminals)
                        ext_customer_profile_idx = random.choice(customer_profile.available_beneficiaries)

                        ext_customer_id = ext_customer_profile.iloc[ext_customer_profile_idx].CUSTOMER_ID
                        ext_iban = ext_customer_profile.iloc[ext_customer_profile_idx].iban
                        ext_bic = ext_customer_profile.iloc[ext_customer_profile_idx].bic
                        ext_name = ext_customer_profile.iloc[ext_customer_profile_idx].fullname
                        ext_country = ext_customer_profile.iloc[ext_customer_profile_idx].country
                        tx_operation_type = "bank_transfer"
                        tx_direction_sample = tx_direction.direction.sample(n=1).iloc[0] # could be a customer profile behaviour
                        tx_status_sample = tx_status.status.sample(n=1).iloc[0]
                        tx_sepa_scheme_sample =  tx_sepa_scheme.scheme.sample(n=1).iloc[0]  
                        tx_code_status_sample = tx_code_status[0]

                        customer_transactions.append([time_tx+day*86400, day,
                                                      customer_profile.CUSTOMER_ID, customer_profile.iban,customer_profile.bic,customer_profile.fullname,customer_profile.country,
                                                      ext_customer_id, ext_iban, ext_bic,ext_name,ext_country,# add other beneficiary info here 
                                                      tx_operation_type, tx_direction_sample, tx_status_sample, tx_sepa_scheme_sample, tx_code_status_sample,
                                                      0, amount])
            
    customer_transactions = pd.DataFrame(customer_transactions, columns=['TX_TIME_SECONDS', 'TX_TIME_DAYS',
     'CUSTOMER_ID', 'IBAN','BIC','FULLNAME','COUNTRY',
     'EXT_CUSTOMER_ID', 'EXT_IBAN', 'EXT_BIC', 'EXT_NAME', 'EXT_COUNTRY', 
     'TX_OPERATION_TYPE', 'TX_DIRECTION','TX_STATUS', 'TX_SEPA_SCHEME', 'TX_CODE_STATUS', 'TERMINAL_ID', 'TX_AMOUNT'])
     
    if len(customer_transactions)>0:
        customer_transactions['TX_DATETIME'] = pd.to_datetime(customer_transactions["TX_TIME_SECONDS"], unit='s', origin=start_date)
        customer_transactions=customer_transactions[['TX_DATETIME', 'CUSTOMER_ID', 'IBAN','BIC','FULLNAME','COUNTRY',
     'EXT_CUSTOMER_ID', 'EXT_IBAN', 'EXT_BIC', 'EXT_NAME', 'EXT_COUNTRY', 
     'TX_OPERATION_TYPE', 'TX_DIRECTION','TX_STATUS', 'TX_SEPA_SCHEME', 'TX_CODE_STATUS', 'TERMINAL_ID', 'TX_AMOUNT','TX_TIME_SECONDS', 'TX_TIME_DAYS']]
    
    return customer_transactions

In [12]:
def generate_dataset(n_customers = 10000, n_terminals = 1000000, nb_days=90, start_date="2022-11-04", r=5):
    
    start_time=time.time()
    customer_profiles_table = generate_customer_profiles_table(n_customers, random_state = 0)
    print("Time to generate customer profiles table: {0:.2}s".format(time.time()-start_time))
    
    start_time=time.time()
    terminal_profiles_table = generate_terminal_profiles_table(n_terminals, random_state = 1)
    print("Time to generate terminal profiles table: {0:.2}s".format(time.time()-start_time))
    
    start_time=time.time()
    x_y_terminals = terminal_profiles_table[['x_terminal_id','y_terminal_id']].values.astype(float)
    customer_profiles_table['available_terminals'] = customer_profiles_table.apply(lambda x : get_list_terminals_within_radius(x, x_y_terminals=x_y_terminals, r=r), axis=1)
    # With Pandarallel
    #customer_profiles_table['available_terminals'] = customer_profiles_table.parallel_apply(lambda x : get_list_closest_terminals(x, x_y_terminals=x_y_terminals, r=r), axis=1)
    customer_profiles_table['nb_terminals']=customer_profiles_table.available_terminals.apply(len)
    print("Time to associate terminals to customers: {0:.2}s".format(time.time()-start_time))
    
    start_time=time.time()
    transactions_df=customer_profiles_table.groupby('CUSTOMER_ID').apply(lambda x : generate_transactions_table(x.iloc[0], start_date=start_date, nb_days=nb_days)).reset_index(drop=True)
    # With Pandarallel
    #transactions_df=customer_profiles_table.groupby('CUSTOMER_ID').parallel_apply(lambda x : generate_transactions_table(x.iloc[0], nb_days=nb_days)).reset_index(drop=True)
    print("Time to generate transactions: {0:.2}s".format(time.time()-start_time))
    
    # Sort transactions chronologically
    transactions_df=transactions_df.sort_values('TX_DATETIME')
    # Reset indices, starting from 0
    transactions_df.reset_index(inplace=True,drop=True)
    transactions_df.reset_index(inplace=True)
    # TRANSACTION_ID are the dataframe indices, starting from 0
    transactions_df.rename(columns = {'index':'TRANSACTION_ID'}, inplace = True)
    
    return (customer_profiles_table, terminal_profiles_table, transactions_df)

In [13]:
def generate_mixed_dataset(n_customers = 10000, n_terminals = 1000000, nb_days=90, start_date="2022-11-04", r=5):
    
    start_time=time.time()
    customer_profiles_table = generate_customer_profiles_table(n_customers, random_state = 0)
    ext_customer_profiles_table = generate_customer_profiles_table(n_customers, random_state = 0) # ext
    print("Time to generate customer profiles table: {0:.2}s".format(time.time()-start_time))
    
    start_time=time.time()
    terminal_profiles_table = generate_terminal_profiles_table(n_terminals, random_state = 1)
    print("Time to generate terminal profiles table: {0:.2}s".format(time.time()-start_time))
    
    start_time=time.time()
    x_y_terminals = terminal_profiles_table[['x_terminal_id','y_terminal_id']].values.astype(float)
    customer_profiles_table['available_terminals'] = customer_profiles_table.apply(lambda x : get_list_terminals_within_radius(x, x_y_terminals=x_y_terminals, r=r), axis=1)
    customer_profiles_table['available_beneficiaries'] = customer_profiles_table.apply(lambda x : get_list_beneficiary_within_date(x,ext_customer_profiles_table), axis=1)
    #customer_profiles_table = pd.concat([customer_profiles_table.drop(columns='available_beneficiaries') , pd.json_normalize(customer_profiles_table['available_beneficiaries'])], axis=1)
    # With Pandarallel
    #customer_profiles_table['available_terminals'] = customer_profiles_table.parallel_apply(lambda x : get_list_closest_terminals(x, x_y_terminals=x_y_terminals, r=r), axis=1)
    customer_profiles_table['nb_terminals']=customer_profiles_table.available_terminals.apply(len)
    print("Time to associate terminals to customers: {0:.2}s".format(time.time()-start_time))
    
    start_time=time.time()
    transactions_df=customer_profiles_table.groupby('CUSTOMER_ID').apply(lambda x : generate_transactions_table(x.iloc[0], start_date=start_date, nb_days=nb_days)).reset_index(drop=True)
    transactions_sepa_df=customer_profiles_table.groupby('CUSTOMER_ID').apply(lambda x : generate_sepa_trx_table(x.iloc[0],ext_customer_profiles_table, start_date=start_date, nb_days=nb_days)).reset_index(drop=True)
    print("Time to generate transactions: {0:.2}s".format(time.time()-start_time))

    # Sort transactions chronologically
    transactions_df=transactions_df.sort_values('TX_DATETIME')
    transactions_sepa_df=transactions_sepa_df.sort_values('TX_DATETIME')

    # Reset indices, starting from 0
    transactions_df.reset_index(inplace=True,drop=True)
    transactions_df.reset_index(inplace=True)
    transactions_sepa_df.reset_index(inplace=True,drop=True)
    transactions_sepa_df.reset_index(inplace=True)

    # TRANSACTION_ID are the dataframe indices, starting from 0
    transactions_df.rename(columns = {'index':'TRANSACTION_ID'}, inplace = True)
    transactions_sepa_df.rename(columns = {'index':'TRANSACTION_ID'}, inplace = True)

    return (customer_profiles_table, terminal_profiles_table, transactions_df,transactions_sepa_df)



In [23]:
start_time = (datetime.datetime.now() - datetime.timedelta(hours=72)).strftime("%Y-%m-%d")
start_time = (datetime.datetime.now() - datetime.timedelta(days=365)).strftime("%Y-%m-%d")

print(start_time)

2021-11-22


In [24]:
end_time = datetime.datetime.now().strftime("%Y-%m-%d")
print(end_time)

2022-11-22


In [16]:
(customer_profiles_table, terminal_profiles_table, transactions_df)=\
    generate_dataset(n_customers = 500, 
                     n_terminals = 10000, 
                     nb_days=7, 
                     start_date=start_time, 
                     r=5)


Time to generate customer profiles table: 1.4s
Time to generate terminal profiles table: 0.04s
Time to associate terminals to customers: 0.13s
Time to generate transactions: 0.92s


In [17]:
%pip install openpyxl

(customer_profiles_table, terminal_profiles_table, transactions_df, transactions_sepa_df )=\
    generate_mixed_dataset(n_customers = 500, 
                     n_terminals = 10000, 
                     nb_days=7, 
                     start_date=start_time, 
                     r=5)

pd.set_option("max_colwidth", 20)
transactions_sepa_df.head(100).to_csv("output.csv")

#dataset = pd.concat([transactions_df, transactions_sepa_df])
#print(dataset.info(verbose=True))
#print(dataset.describe())
#print(dataset.shape)
#print(dataset.columns)


     |████████████████████████████████| 242 kB 1.2 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.
Time to generate customer profiles table: 3.0s
Time to generate terminal profiles table: 0.04s
Time to associate terminals to customers: 0.56s
Time to generate transactions: 6.1s


In [25]:
%pip install openpyxl

(customer_profiles_table, terminal_profiles_table, transactions_df, transactions_sepa_df )=\
    generate_mixed_dataset(n_customers = 5000, 
                     n_terminals = 10000, 
                     nb_days=183, 
                     start_date=start_time, 
                     r=5)

pd.set_option("max_colwidth", 20)
transactions_sepa_df.head(100).to_csv("output.csv")



Note: you may need to restart the kernel to use updated packages.
Time to generate customer profiles table: 2.9e+01s
Time to generate terminal profiles table: 0.04s
Time to associate terminals to customers: 6.0s
Time to generate transactions: 1.1e+03s


In [18]:
def add_frauds(customer_profiles_table, terminal_profiles_table, transactions_df):
    
    # By default, all transactions are genuine
    transactions_df['TX_FRAUD']=0
    transactions_df['TX_FRAUD_SCENARIO']=0
    
    # Scenario 1
    transactions_df.loc[transactions_df.TX_AMOUNT>220, 'TX_FRAUD']=1
    transactions_df.loc[transactions_df.TX_AMOUNT>220, 'TX_FRAUD_SCENARIO']=1
    nb_frauds_scenario_1=transactions_df.TX_FRAUD.sum()
    print("Number of frauds from scenario 1: "+str(nb_frauds_scenario_1))
    
    # Scenario 2
    for day in range(transactions_df.TX_TIME_DAYS.max()):
        
        compromised_terminals = terminal_profiles_table.TERMINAL_ID.sample(n=2, random_state=day)
        
        compromised_transactions=transactions_df[(transactions_df.TX_TIME_DAYS>=day) & 
                                                    (transactions_df.TX_TIME_DAYS<day+28) & 
                                                    (transactions_df.TERMINAL_ID.isin(compromised_terminals))]
                            
        transactions_df.loc[compromised_transactions.index,'TX_FRAUD']=1
        transactions_df.loc[compromised_transactions.index,'TX_FRAUD_SCENARIO']=2
    
    nb_frauds_scenario_2=transactions_df.TX_FRAUD.sum()-nb_frauds_scenario_1
    print("Number of frauds from scenario 2: "+str(nb_frauds_scenario_2))
    
    # Scenario 3
    for day in range(transactions_df.TX_TIME_DAYS.max()):
        
        compromised_customers = customer_profiles_table.CUSTOMER_ID.sample(n=3, random_state=day).values
        
        compromised_transactions=transactions_df[(transactions_df.TX_TIME_DAYS>=day) & 
                                                    (transactions_df.TX_TIME_DAYS<day+14) & 
                                                    (transactions_df.CUSTOMER_ID.isin(compromised_customers))]
        
        nb_compromised_transactions=len(compromised_transactions)
        
        
        random.seed(day)
        index_fauds = random.sample(list(compromised_transactions.index.values),k=int(nb_compromised_transactions/3))
        
        transactions_df.loc[index_fauds,'TX_AMOUNT']=transactions_df.loc[index_fauds,'TX_AMOUNT']*5
        transactions_df.loc[index_fauds,'TX_FRAUD']=1
        transactions_df.loc[index_fauds,'TX_FRAUD_SCENARIO']=3
        
                             
    nb_frauds_scenario_3=transactions_df.TX_FRAUD.sum()-nb_frauds_scenario_2-nb_frauds_scenario_1
    print("Number of frauds from scenario 3: "+str(nb_frauds_scenario_3))
    
    return transactions_df        

In [26]:
import re
%pip install tabulate

pattern = re.compile("^([A-Z][A-Z]+)+$")

def add_scenario_for_frauder():
    return 

def add_sepa_frauds(customer_profiles_table, terminal_profiles_table, transactions_sepa_df):
    
    # By default, all transactions are genuine
    transactions_sepa_df['TX_FRAUD']=0
    transactions_sepa_df['TX_FRAUD_SCENARIO']=0
    
    # Scenario 1 ( amount more than 300)
    transactions_sepa_df.loc[transactions_sepa_df.TX_AMOUNT>5000, 'TX_FRAUD']=1
    transactions_sepa_df.loc[transactions_sepa_df.TX_AMOUNT>5000, 'TX_FRAUD_SCENARIO']=1
    nb_frauds_scenario_1=transactions_sepa_df.TX_FRAUD.sum()
    print("Number of frauds from scenario 1: "+str(nb_frauds_scenario_1))
    
    # Scenario 2 ( country not in the list)
    pattern = r'[A-Z][A-Z]'
    transactions_sepa_df.IBAN= transactions_sepa_df.IBAN.apply(str)
    transactions_sepa_df.loc[transactions_sepa_df.IBAN.str.contains(pattern) != True , 'TX_FRAUD']=1
    transactions_sepa_df.loc[transactions_sepa_df.IBAN.str.contains(pattern) != True , 'TX_FRAUD_SCENARIO']=2
    nb_frauds_scenario_2=transactions_sepa_df.TX_FRAUD.sum()-nb_frauds_scenario_1
    print("Number of frauds from scenario 2: "+str(nb_frauds_scenario_2))

    return transactions_sepa_df


transactions_sepa_df = add_sepa_frauds(customer_profiles_table, terminal_profiles_table, transactions_sepa_df)
transactions_sepa_df


Note: you may need to restart the kernel to use updated packages.
Number of frauds from scenario 1: 377046
Number of frauds from scenario 2: 185792


,TRANSACTION_ID,TX_DATETIME,CUSTOMER_ID,IBAN,BIC,FULLNAME,COUNTRY,EXT_CUSTOMER_ID,EXT_IBAN,EXT_BIC,...,TX_DIRECTION,TX_STATUS,TX_SEPA_SCHEME,TX_CODE_STATUS,TERMINAL_ID,TX_AMOUNT,TX_TIME_SECONDS,TX_TIME_DAYS,TX_FRAUD,TX_FRAUD_SCENARIO
0,0,2022-11-04 00:00:31,1784,FR76443190000000...,PREUFRP1XXX,Clarhe Schroder,FR,2637,Unknown bank cod...,Unknown bank cod...,...,credit,PENDING,SCTR,140005,0,1172.80,31,0,0,0
1,1,2022-11-04 00:01:59,3305,FR76187070000000...,CCBPFRPPXXX,Sayce Theloosen,FR,1728,FR76159300000000...,NATXFRPPPAI,...,credit,VALIDATED,SDDE,140005,0,6252.57,119,0,1,1
2,2,2022-11-04 00:02:52,4495,Bank code exceed...,Bank code exceed...,Babcock Allys,IE,2508,Unknown country-...,Unknown country-...,...,credit,CANCELLED,SCTE,140005,0,840.37,172,0,1,2
3,3,2022-11-04 00:04:21,365,FR76113150000000...,CEPAFRPP131,Bhupinder Spense,FR,1212,FR76188290000000...,CMBRFR2B,...,credit,PENDING,SCTR,140005,0,9444.78,261,0,1,1
4,4,2022-11-04 00:04:50,1452,Unknown country-...,Unknown country-...,Manton Dygert,IN,3181,Bank code exceed...,Bank code exceed...,...,debit,CANCELLED,SCTE,140005,0,3220.80,290,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1794049,1794049,2023-05-05 23:56:54,1767,Unknown bank cod...,Unknown bank cod...,Dogs Calhoun,GF,3843,Bank code exceed...,Bank code exceed...,...,credit,PENDING,SCTE,140005,0,5639.11,15811014,182,1,1
1794050,1794050,2023-05-05 23:57:32,3583,Bank code exceed...,Bank code exceed...,Isabell Kaule,ME,4328,Unknown country-...,Unknown country-...,...,credit,PENDING,SDDR,140005,0,9939.06,15811052,182,1,2
1794051,1794051,2023-05-05 23:58:54,1427,FR76300040000000...,BNPAFRPPIFO,Dorette Alicea,FR,2119,Unknown country-...,Unknown country-...,...,credit,VALIDATED,SCTR,140005,0,57.33,15811134,182,0,0
1794052,1794052,2023-05-05 23:59:25,908,Unknown country-...,Unknown country-...,Polak Jammu,CU,4175,Unknown country-...,Unknown country-...,...,credit,VALIDATED,SCTR,140005,0,2103.99,15811165,182,0,0


In [27]:
#transactions_df = transactions_sepa_df

In [20]:
transactions_df = add_frauds(customer_profiles_table, terminal_profiles_table, transactions_df)
transactions_df

Number of frauds from scenario 1: 186
Number of frauds from scenario 2: 9
Number of frauds from scenario 3: 46


,TRANSACTION_ID,TX_DATETIME,CUSTOMER_ID,TERMINAL_ID,TX_AMOUNT,TX_TIME_SECONDS,TX_TIME_DAYS,TX_FRAUD,TX_FRAUD_SCENARIO
0,0,2022-11-04 00:03:12,53,6793,79.49,192,0,0,0
1,1,2022-11-04 00:04:21,365,6845,157.41,261,0,0,0
2,2,2022-11-04 00:25:01,207,3853,83.70,1501,0,0,0
3,3,2022-11-04 00:32:35,183,5484,68.41,1955,0,0,0
4,4,2022-11-04 00:35:35,25,3308,195.52,2135,0,0,0
...,...,...,...,...,...,...,...,...,...
6947,6947,2022-11-10 23:17:51,183,8550,98.44,602271,6,0,0
6948,6948,2022-11-10 23:30:02,149,429,44.72,603002,6,0,0
6949,6949,2022-11-10 23:32:31,245,5182,85.63,603151,6,0,0
6950,6950,2022-11-10 23:39:29,415,2528,50.26,603569,6,0,0


In [21]:
import json

data = '''{
  "operationType": "bank_transfer",
  "amount": {
    "amount": 6300,
    "currency": "EUR"
  },
  "walletId": 630632,
  "direction": "DEBIT",
  "objectId": 408265455,
  "label": "Bank Transfer (408265455)",
  "externalReference": null,
  "metadata": null,
  "status": "CANCELED",
  "date": {
    "creation": "2020-08-19T06:08:23+02:00",
    "settlement": null
  }
}'''

sample_api = json.loads(data)
sample_df = pd.DataFrame(sample_api)

df = pd.json_normalize(
    sample_api
)

#" https://towardsdatascience.com/how-to-convert-json-into-a-pandas-dataframe-100b2ae1e0d8"
# step transform 

df = df.drop("label",axis=1)
df = df.drop("externalReference",axis=1)
df = df.drop("metadata",axis=1)
df = df.drop("date.settlement",axis=1)
df = df.drop("amount.currency",axis=1)
df['TX_FRAUD'] = 0
df['TX_FRAUD_SCENARIO'] = 0
df['TERMINAL_ID'] = 0
df['TX_TIME_SECONDS'] = 0
df['TX_TIME_DAYS'] = 0

df['DEBTOR'] = 0
df['CREDITOR'] = 0


colNames = {str("walletId"): str("CUSTOMER_ID"), str("objectId"): str("TRANSACTION_ID"), str("amount.amount"):str("TX_AMOUNT"), str("date.creation"):str("TX_DATETIME")}
print(colNames)
df = df.rename(columns=colNames)
print(df)


cols = ['TRANSACTION_ID','TX_DATETIME','CUSTOMER_ID','TERMINAL_ID','TX_AMOUNT','TX_TIME_SECONDS','TX_TIME_DAYS','TX_FRAUD','TX_FRAUD_SCENARIO']
df = df[cols]
df



{'walletId': 'CUSTOMER_ID', 'objectId': 'TRANSACTION_ID', 'amount.amount': 'TX_AMOUNT', 'date.creation': 'TX_DATETIME'}
   operationType  CUSTOMER_ID direction  TRANSACTION_ID    status  TX_AMOUNT  \
0  bank_transfer       630632     DEBIT       408265455  CANCELED       6300   

           TX_DATETIME  TX_FRAUD  TX_FRAUD_SCENARIO  TERMINAL_ID  \
0  2020-08-19T06:08...         0                  0            0   

   TX_TIME_SECONDS  TX_TIME_DAYS  DEBTOR  CREDITOR  
0                0             0       0         0  


,TRANSACTION_ID,TX_DATETIME,CUSTOMER_ID,TERMINAL_ID,TX_AMOUNT,TX_TIME_SECONDS,TX_TIME_DAYS,TX_FRAUD,TX_FRAUD_SCENARIO
0,408265455,2020-08-19T06:08...,630632,0,6300,0,0,0,0


🛠️ Feature Engineering

In [28]:
def is_weekend(tx_datetime):
    
    # Transform date into weekday (0 is Monday, 6 is Sunday)
    weekday = tx_datetime.weekday()
    # Binary value: 0 if weekday, 1 if weekend
    is_weekend = weekday>=5
    
    return int(is_weekend)

transactions_df['TX_DURING_WEEKEND']=transactions_df.TX_DATETIME.apply(is_weekend)

def is_night(tx_datetime):
    
    # Get the hour of the transaction
    tx_hour = tx_datetime.hour
    # Binary value: 1 if hour less than 6, and 0 otherwise
    is_night = tx_hour<=6
    
    return int(is_night)

transactions_df['TX_DURING_NIGHT']=transactions_df.TX_DATETIME.apply(is_night)

In [29]:
def get_customer_spending_behaviour_features(customer_transactions, windows_size_in_days=[1,7,30]):
    
    # Let us first order transactions chronologically
    customer_transactions=customer_transactions.sort_values('TX_DATETIME')
    
    # The transaction date and time is set as the index, which will allow the use of the rolling function 
    customer_transactions.index=customer_transactions.TX_DATETIME
    
    # For each window size
    for window_size in windows_size_in_days:
        
        # Compute the sum of the transaction amounts and the number of transactions for the given window size
        SUM_AMOUNT_TX_WINDOW=customer_transactions['TX_AMOUNT'].rolling(str(window_size)+'d').sum()
        NB_TX_WINDOW=customer_transactions['TX_AMOUNT'].rolling(str(window_size)+'d').count()
    
        # Compute the average transaction amount for the given window size
        # NB_TX_WINDOW is always >0 since current transaction is always included
        AVG_AMOUNT_TX_WINDOW=SUM_AMOUNT_TX_WINDOW/NB_TX_WINDOW
    
        # Save feature values
        customer_transactions['CUSTOMER_ID_NB_TX_'+str(window_size)+'DAY_WINDOW']=list(NB_TX_WINDOW)
        customer_transactions['CUSTOMER_ID_AVG_AMOUNT_'+str(window_size)+'DAY_WINDOW']=list(AVG_AMOUNT_TX_WINDOW)
    
    # Reindex according to transaction IDs
    customer_transactions.index=customer_transactions.TRANSACTION_ID
        
    # And return the dataframe with the new features
    return customer_transactions

transactions_df=transactions_df.groupby('CUSTOMER_ID').apply(lambda x: get_customer_spending_behaviour_features(x, windows_size_in_days=[1,7,30]))
transactions_df=transactions_df.sort_values('TX_DATETIME').reset_index(drop=True)

In [30]:
def get_count_risk_rolling_window(terminal_transactions, delay_period=7, windows_size_in_days=[1,7,30], feature="TERMINAL_ID"):
    
    terminal_transactions=terminal_transactions.sort_values('TX_DATETIME')
    
    terminal_transactions.index=terminal_transactions.TX_DATETIME
    
    NB_FRAUD_DELAY=terminal_transactions['TX_FRAUD'].rolling(str(delay_period)+'d').sum()
    NB_TX_DELAY=terminal_transactions['TX_FRAUD'].rolling(str(delay_period)+'d').count()
    
    for window_size in windows_size_in_days:
    
        NB_FRAUD_DELAY_WINDOW=terminal_transactions['TX_FRAUD'].rolling(str(delay_period+window_size)+'d').sum()
        NB_TX_DELAY_WINDOW=terminal_transactions['TX_FRAUD'].rolling(str(delay_period+window_size)+'d').count()
    
        NB_FRAUD_WINDOW=NB_FRAUD_DELAY_WINDOW-NB_FRAUD_DELAY
        NB_TX_WINDOW=NB_TX_DELAY_WINDOW-NB_TX_DELAY
    
        RISK_WINDOW=NB_FRAUD_WINDOW/NB_TX_WINDOW
        
        terminal_transactions[feature+'_NB_TX_'+str(window_size)+'DAY_WINDOW']=list(NB_TX_WINDOW)
        terminal_transactions[feature+'_RISK_'+str(window_size)+'DAY_WINDOW']=list(RISK_WINDOW)
        
    terminal_transactions.index=terminal_transactions.TRANSACTION_ID
    
    # Replace NA values with 0 (all undefined risk scores where NB_TX_WINDOW is 0) 
    terminal_transactions.fillna(0,inplace=True)
    
    return terminal_transactions

transactions_df=transactions_df.groupby('TERMINAL_ID').apply(lambda x: get_count_risk_rolling_window(x, delay_period=7, windows_size_in_days=[1,7,30], feature="TERMINAL_ID"))
transactions_df=transactions_df.sort_values('TX_DATETIME').reset_index(drop=True)

In [34]:
colNames = {str(col): str(col).upper() for col in transactions_df.columns}
transactions_df = transactions_df.rename(columns=colNames)

#colNames = {str(col): str(col).lower() for col in fraud_labels.columns}
#fraud_labels = fraud_labels.rename(columns=colNames)

In [35]:
print(transactions_df.TX_DATETIME.head(1))
print(transactions_df.TX_DATETIME.tail(1))

,TRANSACTION_ID,TX_DATETIME,CUSTOMER_ID,IBAN,BIC,FULLNAME,COUNTRY,EXT_CUSTOMER_ID,EXT_IBAN,EXT_BIC,...,CUSTOMER_ID_NB_TX_7DAY_WINDOW,CUSTOMER_ID_AVG_AMOUNT_7DAY_WINDOW,CUSTOMER_ID_NB_TX_30DAY_WINDOW,CUSTOMER_ID_AVG_AMOUNT_30DAY_WINDOW,TERMINAL_ID_NB_TX_1DAY_WINDOW,TERMINAL_ID_RISK_1DAY_WINDOW,TERMINAL_ID_NB_TX_7DAY_WINDOW,TERMINAL_ID_RISK_7DAY_WINDOW,TERMINAL_ID_NB_TX_30DAY_WINDOW,TERMINAL_ID_RISK_30DAY_WINDOW
0,0,2022-11-04 00:00:31,1784,FR76443190000000...,PREUFRP1XXX,Clarhe Schroder,FR,2637,Unknown bank cod...,Unknown bank cod...,...,1.0,1172.80,1.0,1172.80,0.0,0.0,0.0,0.0,0.0,0.0
1,1,2022-11-04 00:01:59,3305,FR76187070000000...,CCBPFRPPXXX,Sayce Theloosen,FR,1728,FR76159300000000...,NATXFRPPPAI,...,1.0,6252.57,1.0,6252.57,0.0,0.0,0.0,0.0,0.0,0.0
2,2,2022-11-04 00:02:52,4495,Bank code exceed...,Bank code exceed...,Babcock Allys,IE,2508,Unknown country-...,Unknown country-...,...,1.0,840.37,1.0,840.37,0.0,0.0,0.0,0.0,0.0,0.0
3,3,2022-11-04 00:04:21,365,FR76113150000000...,CEPAFRPP131,Bhupinder Spense,FR,1212,FR76188290000000...,CMBRFR2B,...,1.0,9444.78,1.0,9444.78,0.0,0.0,0.0,0.0,0.0,0.0
4,4,2022-11-04 00:04:50,1452,Unknown country-...,Unknown country-...,Manton Dygert,IN,3181,Bank code exceed...,Bank code exceed...,...,1.0,3220.80,1.0,3220.80,0.0,0.0,0.0,0.0,0.0,0.0


In [36]:

DIR_OUTPUT = "./simulated-data-raw/SEPA/"

if not os.path.exists(DIR_OUTPUT):
    os.makedirs(DIR_OUTPUT)

start_date = datetime.datetime.strptime("2018-04-01", "%Y-%m-%d")

for day in range(transactions_df.TX_TIME_DAYS.max()+1):
    
    transactions_day = transactions_df[transactions_df.TX_TIME_DAYS==day].sort_values('TX_TIME_SECONDS')
    
    date = start_date + datetime.timedelta(days=day)
    filename_output = date.strftime("%Y-%m-%d")+'.pkl'
    
    # Protocol=4 required for Google Colab
    transactions_day.to_pickle(DIR_OUTPUT+filename_output, protocol=4)

In [ ]:
window_len = 4
#window_aggs_df = cc_features.aggregate_activity_by_hour(transactions_df, window_len)

!pip install -U hopsworks --quiet
import hopsworks


You should consider upgrading via the 'C:\Users\Sim\anaconda3\python.exe -m pip install --upgrade pip' command.


ModuleNotFoundError: No module named 'hopsworks'

In [ ]:
project = hopsworks.login()
fs = project.get_feature_store()

Copy your Api Key (first register/login): https://c.app.hopsworks.ai/account/api/generated

Paste it here: ··········
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/3339
Connected. Call `.close()` to terminate connection gracefully.


In [ ]:
#transactions_df.tx_datetime = transactions_df.tx_datetime.values.astype(np.int64)
#fraud_labels.tx_datetime = fraud_labels.tx_datetime.values.astype(np.int64)

def date_to_timestamp(date_obj: datetime)-> int:
    return int(date_obj.timestamp() * 1000)

transactions_df.tx_datetime = transactions_df.tx_datetime.map(lambda x: date_to_timestamp(x))

transactions_df

,transaction_id,tx_datetime,customer_id,terminal_id,tx_amount,tx_time_seconds,tx_time_days,tx_fraud,tx_fraud_scenario,tx_during_weekend,...,customer_id_nb_tx_7day_window,customer_id_avg_amount_7day_window,customer_id_nb_tx_30day_window,customer_id_avg_amount_30day_window,terminal_id_nb_tx_1day_window,terminal_id_risk_1day_window,terminal_id_nb_tx_7day_window,terminal_id_risk_7day_window,terminal_id_nb_tx_30day_window,terminal_id_risk_30day_window
0,0,1667520476000,2,1365,218.05,476,0,0,0,0,...,1.0,218.050000,1.0,218.050000,0.0,0.0,0.0,0.0,0.0,0.0
1,1,1667521805000,360,1611,135.84,1805,0,0,0,0,...,1.0,135.840000,1.0,135.840000,0.0,0.0,0.0,0.0,0.0,0.0
2,2,1667521955000,183,4972,58.00,1955,0,0,0,0,...,1.0,58.000000,1.0,58.000000,0.0,0.0,0.0,0.0,0.0,0.0
3,3,1667522639000,382,2079,20.73,2639,0,0,0,0,...,1.0,20.730000,1.0,20.730000,0.0,0.0,0.0,0.0,0.0,0.0
4,4,1667522751000,381,7869,32.18,2751,0,0,0,0,...,1.0,32.180000,1.0,32.180000,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6560,6560,1668122271000,183,3370,83.46,602271,6,0,0,0,...,20.0,74.953000,20.0,74.953000,0.0,0.0,0.0,0.0,0.0,0.0
6561,6561,1668122434000,305,5791,251.20,602434,6,1,1,0,...,3.0,219.153333,3.0,219.153333,0.0,0.0,0.0,0.0,0.0,0.0
6562,6562,1668122516000,444,9940,19.46,602516,6,0,0,0,...,25.0,16.853600,25.0,16.853600,0.0,0.0,0.0,0.0,0.0,0.0
6563,6563,1668123302000,90,707,30.32,603302,6,0,0,0,...,15.0,148.087333,15.0,148.087333,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
import numpy as np

## Function to reduce the DF size
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

transactions_df = reduce_mem_usage(transactions_df)
#fraud_labels = reduce_mem_usage(fraud_labels)

Mem. usage decreased to  0.31 Mb (73.4% reduction)


In [ ]:
fraud_labels = transactions_df.copy()[["tx_datetime","transaction_id", "tx_fraud"]]

In [ ]:
trans_fg = fs.get_feature_group(name="trans_fraud", version=1)
trans_fg.insert(transactions_df, write_options={"wait_for_job" : False})

labels_fg = fs.get_feature_group(name="transactions_fraud_label", version=1)
labels_fg.insert(fraud_labels, write_options={"wait_for_job" : False})

Uploading Dataframe: 0.00% |          | Rows 0/6565 | Elapsed Time: 00:00 | Remaining Time: ?

Launching offline feature group backfill job...
Backfill Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/3339/jobs/named/trans_fraud_1_offline_fg_backfill/executions


Uploading Dataframe: 0.00% |          | Rows 0/6565 | Elapsed Time: 00:00 | Remaining Time: ?

Launching offline feature group backfill job...
Backfill Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/3339/jobs/named/transactions_fraud_label_1_offline_fg_backfill/executions


(<hsfs.core.job.Job at 0x7fc1cf3dde90>, None)